In [1]:
import pandas;
import seaborn as sb;
import numpy as np;
%matplotlib inline
np.set_printoptions(suppress=True)

In [223]:
def readCSVFile(file):
    data=pandas.read_csv(file,",",header=0, na_values='?', skipinitialspace=True);
    return data;
    pass;
def readTrainData(dataset):    
    return dataset.ix[:,1:-1], dataset.ix[:,-1:];
    pass;

def readTestData(dataset):    
    return dataset.ix[:,1:],dataset.ix[:,0:1];
    pass;

def normalizePhi(unNormalizedPhi,last_col_bias=False):    
    #assuming last column as bias column
    no_of_column=len(unNormalizedPhi[0]);
    phi=np.array(unNormalizedPhi);
    print("Normalizing Phi...");  
    std=phi.std(0);
    mean=phi.mean(0);    
    std[no_of_column-1]=1;
    mean[no_of_column-1]=0;
    #phi_normalize=(phi-mean)/std;    
    
    max_vector=phi.max(axis=0)
    phi_normalize=phi/max_vector;    
    
    print("Normalization done.");
    return phi_normalize;
    pass;

def writeTestData(test_id,ystar,filenumber=0):    
    fo = open("sampleSubmission-"+str(filenumber)+".csv", "w");               
    fo.write("id,salary\n");
    m=len(ystar);
    for i in range(m):
        fo.write(str(test_id[i][0])+","+str(ystar[i][0])+"\n");
    fo.close();
    pass;

def dropColumns(dataframe,colList):
    for c in colList:
        dataframe.drop([c], axis = 1, inplace = True);
    pass;

def addColByCategory(dataset):
    return pandas.get_dummies(dataset);
    pass;

def categoryToNumber(dataset,categoryList):
    for c in categoryList:
        if (c in dataset):            
            dataset[c]=pandas.get_dummies(dataset[c]).values.argmax(1);        
    return dataset;
    pass;
    

def handleCategoryData(dataset,categoryList=None,byNumber=False):
    if(byNumber):
        return categoryToNumber(dataset,categoryList)
    else:
        return addColByCategory(dataset);
def findMostFrequentCount(dataset):
    #arr=dataset;
    #axis = 0
    #u, indices = np.unique(arr, return_inverse=True)
    #print(u);
    #u[np.argmax(np.apply_along_axis(np.bincount, axis, indices.reshape(arr.shape),None, np.max(indices) + 1), axis=axis)]
    x = np.array([0, 1, 1, 3, 2, 1, 2, 3]);
    w=np.bincount(dataset[:,1].astype(int))#work-class: Private
    o=np.bincount(dataset[:,6].astype(int))#occupation: Married-civ-spouse
    c=np.bincount(dataset[:,13].astype(int))#country : US
    pass;

def fillNanValue(dataframe,col,value):
    if (col in dataframe):
        dataframe[col].fillna(value, inplace=True);
    pass;

def imputeUnknowValue(dataframe):
    #by most frequent value;
    fillNanValue(dataframe,"workclass","Private");
    fillNanValue(dataframe,"occupation","Craft-repair");
    fillNanValue(dataframe,"native-country","United-States");
    pass;

def addRemainingCol(colList,dataframe,rowCount):
    i=0;
    for c in colList:
        if( c not in dataframe):
            dataframe.insert(i, c, 0);
        i+=1;
    pass;

In [226]:
class NN:
    logging_enabled=True;
    trained_ds=None;
    trained_output=None;
    no_of_hiddenlayer=0;
    neurons_per_hiddenlayer=0;    
    weights=list();#list of matrix of weights
    layer_output=list();#vector of layers
    layer_delta=list(); 
    layer_error=list(); 
    no_of_iteration=1;
    mean_print_rate=10000;
    percent_print_rate=1000;
    learning_rate=1;
    approching_to_zero=1e-15;
    approching_to_one=1-1e-15;
    lamda=0.001;
    log_dir="log";
    log_folder=None;
    enable_bias_per_hidden=False;
    def __init__(self):
        np.random.seed(1);
        self.log_folder=self.log_dir+"/"+self.getTimestamp();
        self.createDir(self.log_folder);
        pass;
    
    def reInit(self):        
        self.layer_output=[0]*(self.no_of_hiddenlayer+2);
        self.layer_delta=[0]*(self.no_of_hiddenlayer+2);
        self.layer_error=[0]*(self.no_of_hiddenlayer+2);        
        pass;
        
    def createNN(self):
        self.no_of_features=len(self.trained_ds[0]);
        self.no_of_datapoint=len(self.trained_ds); 
        if(self.enable_bias_per_hidden and self.neurons_per_hiddenlayer>1):
            self.neurons_per_hiddenlayer+=1;#adding bias neuron whose o/p will always be 1;
            
        self.reInit();
        self.initWeightMatrix();       
        print("Neural Network Created...");
        pass;
    
    def initWeightMatrix(self):        
        for i in range(self.no_of_hiddenlayer):
            if(i==0):
                m=self.no_of_features;
            else:
                m=self.neurons_per_hiddenlayer;
            w_matrix = 2*np.random.random((m,self.neurons_per_hiddenlayer)) - 1;            
            
            if(self.enable_bias_per_hidden):
                #making weights of bias neuron as zero. So that its o/p is always one.
                w_matrix[:,0]=0;
                
            self.weights.append(w_matrix);
            #print(w_matrix);
            
        #last layer weight: For single output
        if(self.no_of_hiddenlayer==0):
            n=self.no_of_features;
        else:
            n=self.neurons_per_hiddenlayer;
        w_vector=2*np.random.random((n,1)) - 1;
        #print(w_vector);
        self.weights.append(w_vector);
        pass;
        
    def getTimestamp(self):
        import os.path;
        import datetime;
        import time;
        ts = datetime.datetime.fromtimestamp(time.time()).strftime('%d-%m-%Y-%H:%M:%S')
        return ts;

    def createDir(self,directory):
        import os.path;
        if not os.path.exists(directory):
            os.makedirs(directory);
        pass;
    
    def writeWeights(self,filenumber=0):
        fname=self.log_folder+"/weights-"+str(filenumber)+"-";
        m=len(self.weights);
        for i in range(m):
            np.save(fname+""+str(i), nn.weights[i])        
        pass;
    
    def writeWeights(self,filenumber=0):
        fname=self.log_folder+"/weights-"+str(filenumber)+"-";
        m=len(self.weights);
        for i in range(m):
            np.save(fname+""+str(i), self.weights[i])        
        pass;
    
    def loadWeights(self,folder,filename_offset="weights-0-"):
        import os.path;
        w=list();
        i=0;
        file_path=folder+"/"+filename_offset+str(i)+".npy";
        print(file_path);
        while (os.path.isfile(file_path)):
            w.append(np.load(file_path));
            i+=1;
            file_path=folder+"/"+filename_offset+str(i)+".npy";
        self.weights=w;
        return w;
    pass

    def activationFunction(self,x,deriv=False):
        return self.actFunSigmoid(x,deriv);
    
    def actFunSigmoid(self,x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))
    
    def actFunReLu(self,x,deriv=False):
        p = x > 0;            
        if(deriv==True):
            return p.astype(int);
        return x * p;
    
    def forwardPropogation(self,datapoints):        
        #intial layer i.e l0 zeroth layer
        self.layer_output[0]=datapoints;
        for i in range(1,self.no_of_hiddenlayer+2):
            prev_layer=self.layer_output[i-1];
            w=self.weights[i-1]
            sum_l=prev_layer.dot(w);           
            l_i=self.activationFunction(sum_l);
            self.layer_output[i]=l_i;   
            if(self.enable_bias_per_hidden and i<=self.no_of_hiddenlayer):
                #making weights of bias neuron as zero. So that its o/p is always one.
                l_i[:,0]=1;
            #print("l"+str(i)+":",l_i);
        return self.layer_output[self.no_of_hiddenlayer+1];
        pass;
    
    def findError(self):
        self.backPropogation(sel.layer_output);
        return self.getMeanError();
        
    def backPropogation(self,l_output):
        last_layer=len(l_output)-1;            
        
        #(target-output)        
        li=l_output[last_layer];
        error_diff=self.trained_output-li;
        delta=error_diff*self.activationFunction(li,deriv=True)    
        self.layer_delta[last_layer]=delta;
        self.layer_error[last_layer]=error_diff;
        
        for i in range(last_layer-1,0,-1):            
            #i-1 th layer calculation of delta
            error_diff=self.layer_delta[i+1].dot(self.weights[i].T);
            #print("l"+str(i)+"_error:",delta);
            delta=error_diff*self.activationFunction(l_output[i],deriv=True)
            self.layer_delta[i]=delta;
            self.layer_error[i]=error_diff;  
            
        #print("l"+str(i)+"_delta:",delta);        
        pass;
    
    def updateWeights(self,l_delta):
        #print(self.learning_rate);
        for i in range(self.no_of_hiddenlayer,-1,-1): # loop upto 0
            #print(str(i)+"==>a",self.weights[i][0]);            
            #print(str(i)+"==>b",self.layer_output[i].T.dot(l_delta[i+1])[0])
            #print(str(i)+"==>c",(self.lamda*self.weights[i])[0])            
            self.weights[i]+=self.learning_rate*self.layer_output[i].T.dot(l_delta[i+1]);
            if(self.lamda!=0):
                self.weights[i]-=self.learning_rate*-((self.lamda)*self.weights[i]);
            #print("w"+str(i)+":",self.layer_output[i].T.dot(l_delta[i+1]));
    
    
    def getMeanError(self):   
        return np.mean(np.abs(self.layer_error[self.no_of_hiddenlayer+1]));
    
    
    def gradientDescent(self):
        for i in range(self.no_of_iteration):
            self.forwardPropogation(self.trained_ds);
            self.backPropogation(self.layer_output);                       
            if ((i%self.mean_print_rate==0 or i >=self.no_of_iteration-6)):
                 self.log(str(i)+"Error:",self.getMeanError()); 
            if (i%self.percent_print_rate==0 or i >=self.no_of_iteration-6):
                self.log(str(i)+"Percent completed",(i*100)/self.no_of_iteration)
                #self.writeWeights();        
            if(i!=self.no_of_iteration-1):#donot update for last iteration            
                self.updateWeights(self.layer_delta);
        print("Error:",self.getMeanError());
        self.writeWeights();
        pass;
    
    def getThresholdValue(self,result,threshold=0.5):
        return(result>threshold).astype(int);

    def train(self):
         self.gradientDescent();
            
    def log(self,text,data):
        if self.logging_enabled:
            print(text,data);
        pass;
pass;

x = np.array([[0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1]])

y = np.array([[0],[1],[1],[1]])

nn= NN();
#dataset OR

#nn.no_of_hiddenlayer=1;
#nn.neurons_per_hiddenlayer=4
#nn.no_of_iteration=1000;
#nn.learning_rate=1;
#nn.lamda=0;
#nn.mean_print_rate=100;
#nn.percent_print_rate=100;
#nn.enable_bias_per_hidden=True;

#dataset: census income
x=mtx_dummy_tds_norm;
y=mtx_trained_y;
nn.no_of_hiddenlayer=2;
nn.neurons_per_hiddenlayer=30;
nn.no_of_iteration=3001;
nn.learning_rate=0.00001;
nn.lamda=0;
nn.mean_print_rate=20;
nn.percent_print_rate=20;
nn.enable_bias_per_hidden=True;

#Common setting
nn.logging_enabled=True;
nn.trained_ds=x;
nn.trained_output=y;
nn.createNN();
#nn.loadWeights("log/08-04-2017-15:45:54");
nn.train();
result=nn.forwardPropogation(x);
print("predicted",result);
print("Predicted-Th",nn.getThresholdValue(result));
print("Actual",y);
result=nn.forwardPropogation(mtx_dummy_testds_norm);
writeTestData(mtx_test_id,result,filenumber=nn.getMeanError());
writeTestData(mtx_test_id,nn.getThresholdValue(result),filenumber="Threshold"+str(nn.getMeanError()));

Neural Network Created...
0Error: 0.324166898964
0Percent completed 0.0
20Error: 0.343846358486
20Percent completed 0.6664445184938353
40Error: 0.341270492665
40Percent completed 1.3328890369876707
60Error: 0.337926105722
60Percent completed 1.9993335554815062
80Error: 0.334585521208
80Percent completed 2.6657780739753414
100Error: 0.33130856077
100Percent completed 3.332222592469177
120Error: 0.328125825062
120Percent completed 3.9986671109630123
140Error: 0.325056538172
140Percent completed 4.665111629456848
160Error: 0.322111714792
160Percent completed 5.331556147950683
180Error: 0.319296392793
180Percent completed 5.9980006664445185
200Error: 0.316611344107
200Percent completed 6.664445184938354
220Error: 0.314054380061
220Percent completed 7.330889703432189
240Error: 0.311621335981
240Percent completed 7.997334221926025
260Error: 0.309306802965
260Percent completed 8.66377874041986
280Error: 0.307104662422
280Percent completed 9.330223258913696
300Error: 0.305008468907
300Percent 

In [ ]:
nn.train();
result=nn.forwardPropogation(x);
print("predicted",result);
print("Predicted-Th",nn.getThresholdValue(result));
print("Actual",y);
result=nn.forwardPropogation(mtx_dummy_testds_norm);
writeTestData(mtx_test_id,result,filenumber=nn.getMeanError());
writeTestData(mtx_test_id,nn.getThresholdValue(result),filenumber="Threshold"+str(nn.getMeanError()));

0Error: 0.22653795153
0Percent completed 0.0
20Error: 0.22651151005
20Percent completed 0.6664445184938353
40Error: 0.226485153773
40Percent completed 1.3328890369876707
60Error: 0.226458882243
60Percent completed 1.9993335554815062
80Error: 0.226432695011
80Percent completed 2.6657780739753414
100Error: 0.226406591627
100Percent completed 3.332222592469177
120Error: 0.226380571647
120Percent completed 3.9986671109630123
140Error: 0.22635463463
140Percent completed 4.665111629456848
160Error: 0.226328780135
160Percent completed 5.331556147950683
180Error: 0.226303007726
180Percent completed 5.9980006664445185
200Error: 0.226277316971
200Percent completed 6.664445184938354
220Error: 0.226251707439
220Percent completed 7.330889703432189
240Error: 0.226226178702
240Percent completed 7.997334221926025
260Error: 0.226200730335
260Percent completed 8.66377874041986
280Error: 0.226175361916
280Percent completed 9.330223258913696
300Error: 0.226150073027
300Percent completed 9.99666777740753
3

In [224]:
#--settings--
pandas.set_option('display.max_columns', None);
#---init---
dir="data/"
trainFile=dir+"train.csv";
testFile=dir+"kaggle_test_data.csv";
categoryList=["workclass","education","marital-status","occupation","relationship","race","sex","native-country"];
drop_col=['native-country',"race"]
trained_dataset=readCSVFile(trainFile);
trained_data,trained_y=readTrainData(trained_dataset);

test_dataset=readCSVFile(testFile);
test_data,test_id=readTestData(test_dataset);

#droping unrelated-columns
dropColumns(trained_data,drop_col);
dropColumns(test_data,drop_col);

#impute:
imputeUnknowValue(trained_data);
imputeUnknowValue(test_data);

#converting categorical data to point wise data
byNumber=False;
dummy_trained_data=handleCategoryData(trained_data,categoryList,byNumber);
dummy_test_data=handleCategoryData(test_data,categoryList,byNumber);


#adding missing column
trained_columns_name=list(dummy_trained_data.columns.values);
addRemainingCol(trained_columns_name,dummy_test_data,len(trained_data))
test_columns_name=list(dummy_test_data.columns.values);

#converting panda data frame to numpy martix
mtx_dummy_tds=dummy_trained_data.as_matrix(columns=None)
mtx_dummy_testds=dummy_test_data.as_matrix(columns=None)
mtx_trained_y=trained_y.as_matrix(columns=None);
mtx_test_id=test_id.as_matrix(columns=None);

#adding bias column
mtx_dummy_tds=np.column_stack((mtx_dummy_tds,np.ones((len(mtx_dummy_tds),1))))
mtx_dummy_testds=np.column_stack((mtx_dummy_testds,np.ones((len(mtx_dummy_testds),1))))

#normalization
mtx_dummy_tds_norm=normalizePhi(mtx_dummy_tds)
mtx_dummy_testds_norm=normalizePhi(mtx_dummy_testds)


#print(mtx_dummy_tds)
#print(mtx_dummy_tds_norm)
#pandas.get_dummies(trained_data.ix[:,1:2])
print("train",np.shape(mtx_dummy_tds_norm),"test",np.shape(mtx_dummy_testds_norm))

Normalizing Phi...
Normalization done.
Normalizing Phi...
Normalization done.
train (38973, 60) test (6878, 60)


In [188]:
for i in range(105):
    print(trained_columns_name[i]+"======"+test_columns_name[i]);
    if(trained_columns_name[i]!=test_columns_name[i]):
        print("#############");
        

age======age
fnlwgt======fnlwgt
education-num======education-num
capital-gain======capital-gain
capital-loss======capital-loss
hours-per-week======hours-per-week
workclass_Federal-gov======workclass_Federal-gov
workclass_Local-gov======workclass_Local-gov
workclass_Never-worked======workclass_Never-worked
workclass_Private======workclass_Private
workclass_Self-emp-inc======workclass_Self-emp-inc
workclass_Self-emp-not-inc======workclass_Self-emp-not-inc
workclass_State-gov======workclass_State-gov
workclass_Without-pay======workclass_Without-pay
education_10th======education_10th
education_11th======education_11th
education_12th======education_12th
education_1st-4th======education_1st-4th
education_5th-6th======education_5th-6th
education_7th-8th======education_7th-8th
education_9th======education_9th
education_Assoc-acdm======education_Assoc-acdm
education_Assoc-voc======education_Assoc-voc
education_Bachelors======education_Bachelors
education_Doctorate======education_Doctorate
educa

IndexError: list index out of range

In [208]:
dummy_test_data

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,18,103497,10,0,0,30,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,29,227026,9,0,0,40,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,36,212465,13,0,0,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,43,346189,14,0,0,50,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,34,2

### 